In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torchvision.models import resnet50, densenet121, mobilenet_v2, convnext_small
import torchvision
import torch.optim as optim
from torch_funcs import fit, test, get_cifar10_loaders, get_mnist_loaders, FullyConnectedNet, SimpleConvNet
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
batch_size = 32
test_batch_size = 64
epochs = 2
lr = 1e-2
momentum = 0.9
num_classes = 10
log_interval = 300

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'CUDA enabled: {use_cuda}')

start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

CUDA enabled: True


In [3]:
model = mobilenet_v2()
model.classifier[1] = nn.Linear(in_features=1280, out_features=num_classes, bias=True)
train_dl, test_dl = get_cifar10_loaders(batch_size, test_batch_size)
loss_func = F.cross_entropy
opt = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

model = model.to(device)

for epoch in range(1, epochs + 1):

	train_history = fit(model, device, train_dl, loss_func, epoch, optimizer=opt, log_interval=log_interval, silent=False)
	_, accuracy = test(model, device, test_dl, loss_func, silent=False)

Files already downloaded and verified
Files already downloaded and verified
Train Epoch: 1 -> batch: 299 [9568/50000 (19%)]	Loss: 2.615775
Train Epoch: 1 -> batch: 599 [19168/50000 (38%)]	Loss: 2.239391
Train Epoch: 1 -> batch: 899 [28768/50000 (58%)]	Loss: 2.051182
Train Epoch: 1 -> batch: 1199 [38368/50000 (77%)]	Loss: 1.929848
Train Epoch: 1 -> batch: 1499 [47968/50000 (96%)]	Loss: 1.779341

Test set: Average loss: 1.6362, Accuracy: 4110/10000 (41%)

Train Epoch: 2 -> batch: 299 [9568/50000 (19%)]	Loss: 1.658011
Train Epoch: 2 -> batch: 599 [19168/50000 (38%)]	Loss: 1.626790
Train Epoch: 2 -> batch: 899 [28768/50000 (58%)]	Loss: 1.592216
Train Epoch: 2 -> batch: 1199 [38368/50000 (77%)]	Loss: 1.538345
Train Epoch: 2 -> batch: 1499 [47968/50000 (96%)]	Loss: 1.546808

Test set: Average loss: 1.4908, Accuracy: 4602/10000 (46%)



In [9]:
def get_celeba_loader(batch_size, image_size=64):
    transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize(image_size),
        torchvision.transforms.CenterCrop(image_size),
        torchvision.transforms.ToTensor(),
        torchvision.stransforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    ds = torchvision.datasets.ImageFolder(root='../datasets/celeba', transform=transform)
    dl = torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=2)

    return dl


def dcgan_weights_init(model):
    '''
    From the DCGAN paper, the authors specify that all model weights shall be
    randomly initializedfrom a Normal distribution with `mean=0`, `stdev=0.02`
    '''
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)


class Generator(nn.Module):
    def __init__(self, nc, nz, ngf):
        '''
        nz - latent vector size
        nc - number of channels
        ngf - size of feature maps in generator
        '''
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. ``(ngf*8) x 4 x 4``
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. ``(ngf*4) x 8 x 8``
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. ``(ngf*2) x 16 x 16``
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. ``(ngf) x 32 x 32``
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. ``(nc) x 64 x 64``
        )

    def forward(self, input):
        return self.main(input)


class Discriminator(nn.Module):
    def __init__(self, nz, nc, ndf):
        '''
        nz - latent vector size
        nc - number of channels
        ndf - size of feature maps in discriminator
        '''
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # input is ``(nc) x 64 x 64``
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf) x 32 x 32``
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*2) x 16 x 16``
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*4) x 8 x 8``
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*8) x 4 x 4``
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [10]:
nc = 3
nz = 100
ngf = 64
ndf = 64

In [ ]:
netG = Generator(nc, nz, ngf).to(device)
netD = Discriminator(nc, nz, ngf).to(device)